In [1]:
%load_ext autoreload
%autoreload
import os, json
from types import SimpleNamespace
from experiment import run_model
from eval import calculate_stats
import pickle
from datetime import datetime 
import torch
from transformers import BertTokenizer
from sklearn.metrics import classification_report
from util.tools import load_config

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = "cpu"

In [2]:
config_folder = "config/"
conifg = load_config(config_folder)

In [4]:
train_inputs, train_label_ids, train_preds, train_loss, dev_inputs, dev_label_ids, dev_loss, dev_preds = run_model(config, device)

100%|████████████████████████████████████████████████████████████████████████████| 61014/61014 [12:11<00:00, 83.46it/s]
INFO:root:***** Running training *****
INFO:root:  Num examples = 61014
INFO:root:  Batch size = 12
INFO:root:  Num steps = 50840
Epoch:   0%|                                                                                    | 0/10 [00:00<?, ?it/s]

C:\ProgramData\Anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
Epoch:  10%|███████▏                                                                | 1/10 [17:19<2:35:56, 1039.65s/it]


so far epoch accuracy:  0.9532730193070443


Epoch:  20%|██████████████▍                                                         | 2/10 [34:37<2:18:33, 1039.24s/it]


so far epoch accuracy:  0.9875930114399973


...

Epoch:  90%|████████████████████████████████████████████████████████████████▊       | 9/10 [2:35:47<17:18, 1038.41s/it]


so far epoch accuracy:  0.9958206313305143


Epoch: 100%|███████████████████████████████████████████████████████████████████████| 10/10 [2:53:05<00:00, 1038.60s/it]


so far epoch accuracy:  0.9963123217622185
Training complete
Best val Acc: 0.996312


100%|████████████████████████████████████████████████████████████████████████████| 15304/15304 [02:57<00:00, 86.21it/s]
INFO:root:***** Running training *****
INFO:root:  Num examples = 15304
INFO:root:  Batch size = 12
INFO:root:  Num steps = 12750


In [5]:
train_inputs, train_label_ids, train_preds, train_loss, dev_inputs, dev_label_ids, dev_loss, dev_preds

(array([[  101,   152,  2064, ...,     0,     0,     0],
        [  101, 20394,  2556, ...,     0,     0,     0],
        [  101, 16752, 20327, ...,     0,     0,     0],
        ...,
        [  101,  1124,  1108, ...,     0,     0,     0],
        [  101,  1153,  1225, ...,     0,     0,     0],
        [  101,   145,  1828, ...,     0,     0,     0]], dtype=int64),
 array([6, 8, 8, ..., 8, 8, 0], dtype=int64),
 array([6, 8, 8, ..., 8, 8, 0], dtype=int64),
 0.00828191455062979,
 array([[  101,   142, 23206, ...,     0,     0,     0],
        [  101,  2173,  6366, ...,     0,     0,     0],
        [  101,  1284,  5165, ...,     0,     0,     0],
        ...,
        [  101,  1130, 16764, ...,     0,     0,     0],
        [  101,  1760,  6281, ...,     0,     0,     0],
        [  101,  5055,  1240, ...,     0,     0,     0]], dtype=int64),
 array([8, 8, 8, ..., 8, 8, 8], dtype=int64),
 0.058478869990625224,
 array([8, 8, 8, ..., 8, 8, 8], dtype=int64))

In [6]:
print(classification_report(dev_label_ids,dev_preds ))

              precision    recall  f1-score   support

           0       0.97      0.98      0.98      1034
           1       0.96      0.98      0.97      1145
           2       0.98      0.98      0.98      1382
           3       0.98      0.98      0.98      1269
           4       0.96      0.95      0.96       136
           5       0.97      0.96      0.97       954
           6       0.97      0.98      0.98       933
           7       0.92      0.92      0.92       250
           8       1.00      1.00      1.00      8201

    accuracy                           0.99     15304
   macro avg       0.97      0.97      0.97     15304
weighted avg       0.99      0.99      0.99     15304



In [7]:
def save_missed_cases_to_file(file_start_name, dev_preds, dev_label_ids, train_inputs):
    tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)
    missed_cases = []
    for i in range(0,50):
        if dev_label_ids[i] !=  dev_preds[i]:
             missed_cases.append([ dev_preds[i],  dev_label_ids[i] , " ". join (tokenizer.convert_ids_to_tokens(train_inputs[i])) ])

    #Save into a file
    missed_cases_file = config.programsettings.REPORTS_DIR +file_start_name + str(datetime.now()).replace(":", "_").replace(".", "_") + ".pkl"
    with open(missed_cases_file, "wb") as f:
        pickle.dump(missed_cases, f)  
        
save_missed_cases_to_file("BIOBERT_fc_missedcases_" , dev_preds, dev_label_ids, train_inputs)

In [8]:
import numpy as np
np.array(dev_label_ids).shape, np.array(dev_preds).shape, np.array(train_preds).shape, np.array(train_label_ids).shape

((15304,), (15304,), (61014,), (61014,))

In [9]:
%autoreload
train_mcc, train_f1_score, train_df_results, train_label_matches_df = calculate_stats(train_label_ids,train_preds )
dev_mcc, dev_f1_score, dev_df_results, dev_label_matches_df = calculate_stats(dev_label_ids,dev_preds )


 label: [6 8 8 ... 8 8 0]

 preds: [6 8 8 ... 8 8 0]

 label: [8 8 8 ... 8 8 8]

 preds: [8 8 8 ... 8 8 8]


In [10]:
all_experiment_results = []
all_experiment_results.append([str(config), train_loss, dev_loss, train_mcc, train_f1_score,dev_mcc,dev_f1_score, 
                               dev_label_ids, dev_preds,train_label_ids,train_preds  ])
all_experiment_results

[['<__main__.obj object at 0x000002761A16BD60>',
  0.00828191455062979,
  0.058478869990625224,
  0.9951462284544215,
  0.9943279991753811,
  0.9821239659295843,
  0.9702119192708771,
  array([8, 8, 8, ..., 8, 8, 8], dtype=int64),
  array([8, 8, 8, ..., 8, 8, 8], dtype=int64),
  array([6, 8, 8, ..., 8, 8, 0], dtype=int64),
  array([6, 8, 8, ..., 8, 8, 0], dtype=int64)]]

In [11]:
dev_label_matches_df

predicted
labels         matched           
ADE-Drug       False           21
               True           229
Dosage-Drug    False           21
               True           912
Duration-Drug  False            7
               True           129
Form-Drug      False           39
               True           915
Frequency-Drug False           20
               True          1249
Reason-Drug    False           19
               True          1015
Route-Drug     False           24
               True          1121
Strength-Drug  False           29
               True          1353
no relation    False            6
               True          8195

##  Try with different configuration

### Just change model from BIOR to BERT Sequence

In [12]:
config.programsettings.MODEL_NAME = "BERT_Sequence"
config.programsettings.DEBUG_PRINT = 0

In [13]:
train_inputs, train_label_ids, train_preds, train_loss, dev_inputs, dev_label_ids, dev_loss, dev_preds = run_model(config, device)

INFO:root:***** Running training *****
INFO:root:  Num examples = 61014
INFO:root:  Batch size = 12
INFO:root:  Num steps = 50840
INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased.tar.gz from cache at cache/a803ce83ca27fecf74c355673c434e51c265fb8a3e0e57ac62a80e38ba98d384.681017f415dfb33ec8d0e04fe51a619f3f01532ecea04edbfd48c5d160550d9c
INFO:pytorch_pretrained_bert.modeling:extracting archive file cache/a803ce83ca27fecf74c355673c434e51c265fb8a3e0e57ac62a80e38ba98d384.681017f415dfb33ec8d0e04fe51a619f3f01532ecea04edbfd48c5d160550d9c to temp dir C:\Users\pnarsina\AppData\Local\Temp\2\tmp1df9lf5v
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_v

C:\ProgramData\Anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
Epoch:  10%|███████▏                                                                | 1/10 [20:09<3:01:25, 1209.48s/it]


so far epoch accuracy:  0.9329662044776609


Epoch:  20%|██████████████▍                                                         | 2/10 [40:19<2:41:17, 1209.64s/it]


so far epoch accuracy:  0.9840692300127839


...

Epoch:  90%|████████████████████████████████████████████████████████████████▊       | 9/10 [3:01:32<20:10, 1210.36s/it]


so far epoch accuracy:  0.9952797718556397


Epoch: 100%|███████████████████████████████████████████████████████████████████████| 10/10 [3:21:43<00:00, 1210.36s/it]


so far epoch accuracy:  0.9959189694168551
Training complete
Best val Acc: 0.995919


INFO:root:***** Running training *****
INFO:root:  Num examples = 15304
INFO:root:  Batch size = 12
INFO:root:  Num steps = 12750


In [14]:

train_mcc, train_f1_score, train_df_results, train_label_matches_df = calculate_stats(train_label_ids,train_preds )
dev_mcc, dev_f1_score, dev_df_results, dev_label_matches_df = calculate_stats(dev_label_ids,dev_preds)


 label: [0 1 8 ... 3 8 8]

 preds: [0 1 8 ... 3 8 8]

 label: [8 8 1 ... 3 8 3]

 preds: [8 8 1 ... 3 8 3]


In [15]:
all_experiment_results.append([str(config), train_loss, dev_loss, train_mcc, train_f1_score,dev_mcc,dev_f1_score, 
                               dev_label_ids, dev_preds,train_label_ids,train_preds  ])
all_experiment_results

[['<__main__.obj object at 0x000002761A16BD60>',
  0.00828191455062979,
  0.058478869990625224,
  0.9951462284544215,
  0.9943279991753811,
  0.9821239659295843,
  0.9702119192708771,
  array([8, 8, 8, ..., 8, 8, 8], dtype=int64),
  array([8, 8, 8, ..., 8, 8, 8], dtype=int64),
  array([6, 8, 8, ..., 8, 8, 0], dtype=int64),
  array([6, 8, 8, ..., 8, 8, 0], dtype=int64)],
 ['<__main__.obj object at 0x000002761A16BD60>',
  0.00937360911897523,
  0.054943587886067645,
  0.9948568712922901,
  0.9936352782576474,
  0.9821243573367711,
  0.9704151639693746,
  array([8, 8, 1, ..., 3, 8, 3], dtype=int64),
  array([8, 8, 1, ..., 3, 8, 3], dtype=int64),
  array([0, 1, 8, ..., 3, 8, 8], dtype=int64),
  array([0, 1, 8, ..., 3, 8, 8], dtype=int64)]]

In [16]:
dev_label_matches_df

predicted
labels         matched           
ADE-Drug       False           29
               True           221
Dosage-Drug    False           21
               True           912
Duration-Drug  False            4
               True           132
Form-Drug      False           33
               True           921
Frequency-Drug False           20
               True          1249
Reason-Drug    False           16
               True          1018
Route-Drug     False           21
               True          1124
Strength-Drug  False           35
               True          1347
no relation    False            7
               True          8194

In [17]:

all_model_results_pickle_file = config.programsettings.REPORTS_DIR + "multi_model_experiment_results_" + str(datetime.now()).replace(":", "_").replace(".", "_") + ".pkl"
with open(all_model_results_pickle_file, "wb") as f:
    pickle.dump(all_experiment_results, f)  


In [18]:
print(classification_report(dev_label_ids,dev_preds ))

              precision    recall  f1-score   support

           0       0.97      0.98      0.98      1034
           1       0.97      0.98      0.97      1145
           2       0.99      0.97      0.98      1382
           3       0.99      0.98      0.98      1269
           4       0.97      0.97      0.97       136
           5       0.97      0.97      0.97       954
           6       0.97      0.98      0.97       933
           7       0.92      0.88      0.90       250
           8       1.00      1.00      1.00      8201

    accuracy                           0.99     15304
   macro avg       0.97      0.97      0.97     15304
weighted avg       0.99      0.99      0.99     15304



In [19]:
# Save missed cases.
save_missed_cases_to_file("BERT_sequential_missedcases_" , dev_preds, dev_label_ids, train_inputs)
